In [1]:
import argparse
import pathlib
import random
import sys
import warnings

import numpy as np
import pandas as pd
import pycytominer.aggregate

# Suppress all RuntimeWarnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
import itertools

sys.path.append("../utils")
from mAP_utils import run_mAP_across_time

# check if in a jupyter notebook
try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False

In [2]:
data_file_path = pathlib.Path(
    "../../data/CP_feature_select/profiles/features_selected_profile.parquet"
).resolve(strict=True)
# load the dose information


pathlib.Path("../results").mkdir(exist_ok=True)
# output filepath
output_file_path = pathlib.Path("../results/mAP_across_channels.parquet").resolve()
df = pd.read_parquet(data_file_path)

df.head()

,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,...,Nuclei_Texture_DifferenceVariance_CL_488_1_3_00_256,Nuclei_Texture_DifferenceVariance_CL_488_2_3_01_256,Nuclei_Texture_DifferenceVariance_CL_561_3_02_256,Nuclei_Texture_InverseDifferenceMoment_CL_488_1_3_01_256,Nuclei_Texture_InverseDifferenceMoment_CL_561_3_01_256,Nuclei_Texture_InverseDifferenceMoment_DNA_3_00_256,Nuclei_Texture_SumAverage_CL_488_1_3_03_256,Nuclei_Texture_SumAverage_CL_488_2_3_03_256,Nuclei_Texture_SumAverage_CL_561_3_02_256,Nuclei_Texture_SumAverage_DNA_3_00_256
0,1,D-02,192,Staurosporine,0.0,negative,1,0001,0.0,2,...,1.439092,0.891046,1.500406,0.587907,0.617193,-3.721488,-0.232102,0.381277,-0.044157,-3.411162
1,1,D-02,192,Staurosporine,0.0,negative,1,0001,0.0,23,...,-1.048757,0.629287,0.600013,-0.178073,0.464530,0.297776,0.356853,0.381277,0.040152,0.158581
2,1,D-02,192,Staurosporine,0.0,negative,1,0001,0.0,31,...,0.480107,0.891046,0.380827,0.448919,0.390155,0.297776,-0.232102,0.381277,0.078288,0.158581
3,1,D-02,192,Staurosporine,0.0,negative,1,0001,0.0,35,...,-0.616834,0.891046,0.863686,0.217271,0.565594,0.297776,-0.030175,0.381277,-0.002435,0.158581
4,1,D-02,192,Staurosporine,0.0,negative,1,0001,0.0,40,...,1.439092,-1.464788,-0.659766,0.587907,0.104220,0.297776,-0.232102,-1.424332,0.471232,0.158581


In [3]:
metadata_cols = [cols for cols in df.columns if "Metadata" in cols]
features_cols = [cols for cols in df.columns if "Metadata" not in cols]
features_cols = features_cols
aggregate_df = pycytominer.aggregate(
    population_df=df,
    strata=["Metadata_Well", "Metadata_Time"],
    features=features_cols,
    operation="median",
)
metadata_df = df[metadata_cols]
metadata_df = metadata_df.drop_duplicates(subset=["Metadata_Well", "Metadata_Time"])
metadata_df = metadata_df.reset_index(drop=True)
aggregate_df = pd.merge(
    metadata_df, aggregate_df, on=["Metadata_Well", "Metadata_Time"]
)
print(aggregate_df.shape)
aggregate_df.head()

(390, 847)


,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,...,Nuclei_Texture_DifferenceVariance_CL_488_1_3_00_256,Nuclei_Texture_DifferenceVariance_CL_488_2_3_01_256,Nuclei_Texture_DifferenceVariance_CL_561_3_02_256,Nuclei_Texture_InverseDifferenceMoment_CL_488_1_3_01_256,Nuclei_Texture_InverseDifferenceMoment_CL_561_3_01_256,Nuclei_Texture_InverseDifferenceMoment_DNA_3_00_256,Nuclei_Texture_SumAverage_CL_488_1_3_03_256,Nuclei_Texture_SumAverage_CL_488_2_3_03_256,Nuclei_Texture_SumAverage_CL_561_3_02_256,Nuclei_Texture_SumAverage_DNA_3_00_256
0,1,D-02,192,Staurosporine,0.0,negative,1,0001,0.0,2,...,0.082244,0.391366,-0.180506,0.377368,0.321500,0.297776,-0.150069,0.381277,0.192643,0.158581
1,1,D-02,187,Staurosporine,0.0,negative,3,0001,2.0,1,...,0.200925,-0.407160,-0.503826,0.368764,0.182390,0.297776,-0.190034,0.381277,0.240063,0.158581
2,1,D-02,182,Staurosporine,0.0,negative,4,0001,3.0,2,...,0.240701,-0.935020,-0.572322,0.365262,0.155922,0.297776,-0.198185,0.381277,0.252495,0.158581
3,1,D-02,184,Staurosporine,0.0,negative,5,0001,4.0,1,...,0.115539,-1.215330,-0.615515,0.366907,0.134764,0.297776,-0.198011,0.381277,0.271527,0.158581
4,1,D-02,187,Staurosporine,0.0,negative,6,0001,5.0,1,...,0.155876,-1.464788,-0.676167,0.347495,0.099809,0.297776,-0.211514,-1.424332,0.312376,0.158581


In [4]:
# get the non metadata columns
metadata_cols = [cols for cols in aggregate_df.columns if "Metadata" in cols]
features_cols = [cols for cols in aggregate_df.columns if "Metadata" not in cols]
# drop features that contain AreaShape in the feature name
non_feature_cols = [cols for cols in features_cols if "AreaShape" in cols]
non_feature_cols = (
    non_feature_cols
    + [col for col in features_cols if "Location" in col]
    + [col for col in features_cols if "Neighbors" in col]
)
correlation_cols = [col for col in features_cols if "Correlation" in col]
features_cols = [cols for cols in features_cols if cols not in non_feature_cols]
features_cols = [cols for cols in features_cols if cols not in correlation_cols]
df = pd.DataFrame(features_cols, columns=["features"])

df[
    [
        "Compartment",
        "Feature_type",
        "Measurement",
        "Channel",
        "misc1",
        "misc2",
        "misc3",
        "misc4",
        "misc5",
    ]
] = df["features"].str.split("_", expand=True)
# Update Channel column where it contains "CL"

df.loc[df["Channel"].str.contains("CL", na=False), "Channel"] = (
    df["Channel"] + "_" + df["misc1"]
)
# Update Channel column where it contains "CL_488"
df.loc[df["Channel"].str.contains("CL_488", na=False), "Channel"] = (
    df["Channel"] + "_" + df["misc2"]
)
df.drop(columns=["misc1", "misc2", "misc3", "misc4", "misc5"], inplace=True)
df.head()

,features,Compartment,Feature_type,Measurement,Channel
0,Cytoplasm_Granularity_1_CL_488_1,Cytoplasm,Granularity,1,CL_488_1
1,Cytoplasm_Granularity_1_CL_488_2,Cytoplasm,Granularity,1,CL_488_2
2,Cytoplasm_Granularity_1_CL_561,Cytoplasm,Granularity,1,CL_561
3,Cytoplasm_Granularity_1_DNA,Cytoplasm,Granularity,1,DNA
4,Cytoplasm_Intensity_IntegratedIntensityEdge_CL...,Cytoplasm,Intensity,IntegratedIntensityEdge,CL_488_2


In [5]:
# create a dictionary of loadable features for each channel
loadable_features = {}
for channel in df.Channel.unique():
    channel_df = df.query("Channel == @channel")
    loadable_features[channel] = channel_df.features.tolist()
loadable_features["None"] = non_feature_cols
loadable_features["All"] = [
    cols for cols in aggregate_df.columns if "Metadata" not in cols
]

In [6]:
unique_channels = df["Channel"].unique().tolist()
# unique_channels = unique_channels + ['None']
# channel combinations
channel_combinations = []
for i in range(1, len(unique_channels) + 1):
    channel_combinations.extend(list(itertools.combinations(unique_channels, i)))
channel_combinations = [list(comb) for comb in channel_combinations]
channel_combinations = channel_combinations + [["None"]]
channel_combinations

[['CL_488_1'],
 ['CL_488_2'],
 ['CL_561'],
 ['DNA'],
 ['CL_488_1', 'CL_488_2'],
 ['CL_488_1', 'CL_561'],
 ['CL_488_1', 'DNA'],
 ['CL_488_2', 'CL_561'],
 ['CL_488_2', 'DNA'],
 ['CL_561', 'DNA'],
 ['CL_488_1', 'CL_488_2', 'CL_561'],
 ['CL_488_1', 'CL_488_2', 'DNA'],
 ['CL_488_1', 'CL_561', 'DNA'],
 ['CL_488_2', 'CL_561', 'DNA'],
 ['CL_488_1', 'CL_488_2', 'CL_561', 'DNA'],
 ['None']]

In [7]:
channel_combo_output_dict = {}
# loop through the channel combinations and shuffle
for shuffle in [False, True]:
    for channel_combination in channel_combinations:
        features_to_load = []
        features_to_load.append(metadata_cols)
        # rename the all channels to all
        if len(channel_combination) == 5:
            channel_combination = ["All"]
        elif channel_combination == ["None"]:
            features_to_load.append(loadable_features["None"])
        else:
            for channel in channel_combination:
                features_to_load.append(loadable_features[channel])
            features_to_load.append(loadable_features["None"])
        # flatten the list
        features_to_load = list(itertools.chain(*features_to_load))
        temporary_df = aggregate_df[features_to_load]
        # shuffle the data
        if shuffle == True:
            print("shuffled")
            shuffle_status = "shuffled"
            random.seed(0)
            # permutate the data
            for col in temporary_df.columns:
                if "Metadata_Time" in col or "Metadata_dose" in col:
                    continue
                else:
                    temporary_df[col] = np.random.permutation(temporary_df[col])
        else:
            print("not shuffled")
            shuffle_status = "non_shuffled"

        # run mAP with the 0 dose as the reference
        dict_of_map_dfs = run_mAP_across_time(
            temporary_df,
            seed=0,
            time_column="Metadata_Time",
            reference_column_name="Metadata_dose",
            reference_group=aggregate_df["Metadata_dose"].min(),
        )
        # concat and rename the columns
        df = pd.concat(dict_of_map_dfs.values(), keys=dict_of_map_dfs.keys())
        df.reset_index(inplace=True)
        df.rename(
            columns={
                "level_0": "Metadata_Time",
            },
            inplace=True,
        )
        df["Channel"] = "_".join(channel_combination)
        df["shuffle"] = shuffle_status
        channel_combo_output_dict[
            f"{'_'.join(channel_combination)}_{shuffle_status}"
        ] = df

not shuffled


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

In [8]:
final_df = pd.concat(
    channel_combo_output_dict.values(), keys=channel_combo_output_dict.keys()
)
final_df.reset_index(inplace=True, drop=True)
# save the output to a file
final_df.to_parquet(output_file_path)
final_df.head()

,Metadata_Time,Metadata_dose,Metadata_reference_index,mean_average_precision,indices,p_value,corrected_p_value,below_p,below_corrected_p,-log10(p-value),Channel,shuffle
0,0.0,0.61,-1,1.0,"[0, 8, 14]",9.999990e-07,9.999990e-07,True,True,6.0,CL_488_1,non_shuffled
1,0.0,1.22,-1,1.0,"[9, 19, 22]",9.999990e-07,9.999990e-07,True,True,6.0,CL_488_1,non_shuffled
2,0.0,2.44,-1,1.0,"[3, 21, 26]",9.999990e-07,9.999990e-07,True,True,6.0,CL_488_1,non_shuffled
3,0.0,4.88,-1,1.0,"[13, 17, 25]",9.999990e-07,9.999990e-07,True,True,6.0,CL_488_1,non_shuffled
4,0.0,9.77,-1,1.0,"[6, 7, 18]",9.999990e-07,9.999990e-07,True,True,6.0,CL_488_1,non_shuffled
